In [1]:
# Own Libraries
from utils.plotting import plot_image_and_similar
from utils.data import DataManager
from utils.metrics import prec_recall, iou_score, f1_dice
from utils.similarity import Similarity
from utils.image_processing import image_to_windows, get_3d_norm_histogram, calculate_histograms,text_removal,text_reading

# 3rd Party Libraries
from skimage.io import imread
from skimage.color import rgb2gray
from typing import Tuple, List
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import pickle
import ml_metrics as metrics
import pytesseract
import re
import textdistance

In [2]:
def denoise_images (images, method='nlmeans', ksize=(3,3)):
    
    (lpfw,lpfh) = ksize
    (gfw,gfh) = ksize
    
    new_images = []
    for img in tqdm(images):
        
        if method == 'low_pass':
            lowPassFilter = np.ones((lpfw,lpfh))*1/(lpfw*lpfh)
            denoised_img = cv2.filter2D(img,-1,lowPassFilter)
        elif method == 'gauss':
            #gaussianFilter = gaussFilter((gfw,gfh),1)
            #gaussian_image = cv2.filter2D(img,-1,gaussianFilter)
            denoised_img = cv2.GaussianBlur(img,ksize,0)

        elif method == 'average':
            denoised_img = cv2.blur(img,ksize)
        
        elif method == 'nlmeans':
            denoised_img = cv2.fastNlMeansDenoisingColored(img, None, 15, 15, 13, 21)
            
        elif method == 'median':
            denoised_img = cv2.medianBlur(img,ksize[0])
        else:
            return None 
        
        new_images.append(denoised_img)
        
    return new_images

In [3]:
data_manager = DataManager()
db, db_files = data_manager.load_data(folder = "./data/BBDD/", extension = ".jpg", desc = "Loading BBDD Data...")
qsd1_w2, qsd1_w2_files = data_manager.load_data(folder = "./data/qsd1_w2/", extension = ".jpg", desc = "Loading qsd1_w2 Data...")

qsd1_w3, qsd1_w3_files = data_manager.load_data(folder = "./data/qst1_w3/", extension = ".jpg", desc = "Loading qsd1_w3 Data...")
qsd2_w3, qsd2_w3_files = data_manager.load_data(folder = "./data/qst2_w3/", extension = ".jpg", desc = "Loading qsd2_w3 Data...")

Loading qsd1_w2 Data...:  53%|█████████████████████████████▊                          | 16/30 [00:00<00:00, 147.18it/s]

./data/BBDD/ read: 287 images


Loading qsd1_w3 Data...:  48%|██████████████████████████▉                             | 24/50 [00:00<00:00, 175.65it/s]

./data/qsd1_w2/ read: 30 images


Loading qsd2_w3 Data...:  53%|█████████████████████████████▊                          | 16/30 [00:00<00:00, 104.17it/s]

./data/qst1_w3/ read: 50 images


Loading qsd2_w3 Data...: 100%|█████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 37.46it/s]


./data/qst2_w3/ read: 30 images


In [4]:
print(len(qsd1_w3_files))
print(qsd2_w3_files)

50
('./data/qst2_w3/00000.jpg', './data/qst2_w3/00001.jpg', './data/qst2_w3/00002.jpg', './data/qst2_w3/00003.jpg', './data/qst2_w3/00004.jpg', './data/qst2_w3/00005.jpg', './data/qst2_w3/00006.jpg', './data/qst2_w3/00007.jpg', './data/qst2_w3/00008.jpg', './data/qst2_w3/00009.jpg', './data/qst2_w3/00010.jpg', './data/qst2_w3/00011.jpg', './data/qst2_w3/00012.jpg', './data/qst2_w3/00013.jpg', './data/qst2_w3/00014.jpg', './data/qst2_w3/00015.jpg', './data/qst2_w3/00016.jpg', './data/qst2_w3/00017.jpg', './data/qst2_w3/00018.jpg', './data/qst2_w3/00019.jpg', './data/qst2_w3/00020.jpg', './data/qst2_w3/00021.jpg', './data/qst2_w3/00022.jpg', './data/qst2_w3/00023.jpg', './data/qst2_w3/00024.jpg', './data/qst2_w3/00025.jpg', './data/qst2_w3/00026.jpg', './data/qst2_w3/00027.jpg', './data/qst2_w3/00028.jpg', './data/qst2_w3/00029.jpg')


In [5]:
BBDD_texts = data_manager.load_text(folder = "./data/BBDD/", extension = ".txt", desc = "Loading BBDD Text...")
print(BBDD_texts)

Loading BBDD Text...: 100%|████████████████████████████████████████████████████████| 287/287 [00:00<00:00, 2459.64it/s]


[('./data/BBDD/bbdd_00000.txt', "('Victor Perez-Porros', 'Des-li-zan-tes')\n"), ('./data/BBDD/bbdd_00001.txt', "('Hugo Demarco', 'Reflecting Room')\n"), ('./data/BBDD/bbdd_00002.txt', '\n'), ('./data/BBDD/bbdd_00003.txt', "('Edvard Munch', 'Youth')\n"), ('./data/BBDD/bbdd_00004.txt', "('Martin Carral', 'Ciclo espacial XXIV')\n"), ('./data/BBDD/bbdd_00005.txt', "('Mario Pasqualotto', 'Els 72 noms de Deu')\n"), ('./data/BBDD/bbdd_00006.txt', "('Gerard Sala', 'S/t')\n"), ('./data/BBDD/bbdd_00007.txt', "('Yago Hortal', 'Kl 45')\n"), ('./data/BBDD/bbdd_00008.txt', "('Josep Guinovart', 'S/t')\n"), ('./data/BBDD/bbdd_00009.txt', '\n'), ('./data/BBDD/bbdd_00010.txt', "('Agusti Puig', 'Sota la lleona')\n"), ('./data/BBDD/bbdd_00011.txt', "('Joan Hernandez Pijuan', 'Serie violetes i verds')\n"), ('./data/BBDD/bbdd_00012.txt', "('Regina Gimenez', 'Navegants')\n"), ('./data/BBDD/bbdd_00013.txt', "('Pere Santilari', 'Berenar de pastisseria')\n"), ('./data/BBDD/bbdd_00014.txt', '\n'), ('./data/BBDD/

In [6]:
BBDD_titles = data_manager.extract_title(data=BBDD_texts,index=1,desc='Extracting titles from DDBB...')
BBDD_authors = data_manager.extract_title(data=BBDD_texts,index=0,desc='Extracting authors from DDBB...')


Extracting authors from DDBB...: 100%|████████████████████████████████████████████| 287/287 [00:00<00:00, 71935.30it/s]


In [7]:
print(BBDD_authors)

['Victor Perez-Porros', 'Hugo Demarco', '', 'Edvard Munch', 'Martin Carral', 'Mario Pasqualotto', 'Gerard Sala', 'Yago Hortal', 'Josep Guinovart', '', 'Agusti Puig', 'Joan Hernandez Pijuan', 'Regina Gimenez', 'Pere Santilari', '', 'Josep Ginovart', 'Corneille', '', '', '', 'Antoni Llena', 'Josep Cisquella', 'Modest Cuixart', 'Per Krohg', 'Joan Pere Viladecans', 'Josep Guinovart', 'Victor Perez-Porro', 'Marco Palazzi', 'Robert Llimos', '', 'Francesca Llopis', 'Pablo Picasso', 'Perejaume', '', 'Jose Luis Pascual', 'Sergi Barnils', '', 'Agusti Puig', '', 'Alfred Figueras', 'Paul Klee', 'Robert Llimos', 'Enrique Barro', 'Narcis Comadira', 'Josep Segu', 'Joan Pere Viladecans', 'Lluis Lleo', 'Xavier Valls', 'Ramiro Fernandez Saus', '', 'Jordi Fulla', 'Alfred Figueras', 'Pere Santilari', 'Alfons Borrell', 'Anna-Eva Bergman', 'Alfons Borrell', '', 'Paul Klee', 'Alfred Figueras', '', 'Jens Ferdinand Willumsen', '', '', 'Edvard Munch', 'Julio Vaquero', '', '', 'Ramon Enrich', 'Jordi Isern', 'Edv

In [6]:
# Mention the installed location of Tesseract-OCR in your system
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract'
qsd1_w3_denoised = denoise_images (qsd1_w3,method='nlmeans', ksize=(3,3))
qsd2_w3_denoised = denoise_images (qsd2_w3,method='nlmeans', ksize=(3,3))
detected_titles = text_reading(data=qsd1_w2,num_images=1,desc='Reading titles from images')
detected_authors_1 = text_reading(data=qsd1_w3_denoised,num_images=1,desc='Reading authors from images')
detected_authors_2 = text_reading(data=qsd2_w3_denoised,num_images=2,desc='Reading authors from images')
     

Reading authors from images: 100%|█████████████████████████████████████████████████████| 30/30 [00:21<00:00,  1.41it/s]


In [23]:
print(detected_authors_1)
print(detected_authors_2)

[' ', ' LIUIS Marsans        RATA NNNNNEAATNATNONVTTTT ANTAL TLALANL Ly hy', ' Perejaume', 'Alfred Figueras', 'A Koy bale exopate', ' Josep Cisquella ', 'Modest Culxart', ' Joaquim Chancho ', 'a Unknown ', 'Antoni Tapies - -', 'B Alfons Borrellff', ' ', 'Alfred Figueras', 'Lluis Lleo', ' ', '-Edvard Munch', 'Xavier Valls', 'KTrONg ', '', 'Josep Baques', 'Joan Hernandez Pijuan', 'f Modest Cuixart EB', 'ps kamon Enrich ', 'Alfred Figueras', 'Alfred Figueras', '', ' Unknown ', 'Per Krohg', 'Francese Artigau', 'iXevi Vilaro', '-Alfred Figueras -', 'Alfred Figueras ', '    Paul Klec ', '4 Unknown i', 'Peter Halley Yago Hortal', 'Wy Antoni Llena ', 'Francesc Artigau', 'ArranzBravo ', 'Francesca Llopis', ' Josep Segu  wal  ', 'a  Ramon Enrich zc', 'mPere Santilari i', 'SAnders Svarstadf', 'Alfred Figueras', 'Alfred Figueras', ' Joan Ponc ', ' ', ' Sergi Barnils ', 'y Sergi Barnils J', '']
[(' ', 'Pablo Picasso '), ('a Alfons Borrell', 'Alfons Borrell '), ('Alfred Figueras ', 'pAlfred Figueras

In [25]:
def save_txt(files:list(),num_images:int):
    if num_images==1:
        for i in range(0,len(files)):
            if i<10:
                file_num = './data/results/0000'
            if i>=10:
                file_num = './data/results/000'
            with open(file_num +str(i)+'.txt', 'w') as f:
                f.write(detected_authors_2[i])
                
    if num_images==2: 
        for i in range(0,len(files)):
            if i<10:
                file_num = './data/results_2/0000'
            if i>=10:
                file_num = './data/results_2/000'
            with open(file_num +str(i)+'.txt', 'w') as f:
                f.write(detected_authors_2[i][0])
                f.write('\n')
                f.write(detected_authors_2[i][1]) if num_images==2: 
    if num_images ==3:
        for i in range(0,len(files)):
            if i<10:
                file_num = './data/results_3/0000'
            if i>=10:
                file_num = './data/results_3/000'
            with open(file_num +str(i)+'.txt', 'w') as f:
                f.write(detected_authors_2[i][0])
                f.write('\n')
                f.write(detected_authors_2[i][1])
                f.write('\n')
                f.write(detected_authors_2[i][2])
                


In [13]:
#_, contours_1 = text_removal(data=qsd1_w3_denoised,num_images=1,method='better' , desc='Bounding boxes from images')
#_, contours_2 = text_removal(data=qsd2_w3_denoised,num_images=2,method='better' , desc='Bounding boxes from images')

print(contours_2)
def contours_to_text_box(contours:np.ndarray, num_boxes:int):
    if num_boxes==1:
        for i in range(0,len(contours)):
            contours[i][2] = contours_2[i][2] + contours[i][0]
            contours[i][3] = contours_2[i][3] + contours[i][1]
    if num_boxes==2:
        for i in range(0,len(contours)):
            contours[i][0][2] = contours[i][0][2] + contours[i][0][0]
            contours[i][0][3] = contours[i][0][3] + contours[i][0][1]
            contours[i][1][2] = contours[i][1][2] + contours[i][1][0]
            contours[i][1][3] = contours[i][1][3] + contours[i][1][1]
    if num_boxes==3:
        for i in range(0,len(contours)):
            contours[i][0][2] = contours[i][0][2] + contours[i][0][0]
            contours[i][0][3] = contours[i][0][3] + contours[i][0][1]
            contours[i][1][2] = contours[i][1][2] + contours[i][1][0]
            contours[i][1][3] = contours[i][1][3] + contours[i][1][1]
            contours[i][2][2] = contours[i][2][2] + contours[i][2][0]
            contours[i][2][3] = contours[i][2][3] + contours[i][2][1]

    return contours

[[[549, 1653, 552, 49], [1568, 1647, 21, 11]], [[199, 548, 430, 45], [0, 0, 0, 0]], [[2770, 2022, 600, 86], [0, 0, 0, 0]], [[68, 464, 494, 69], [0, 0, 0, 0]], [[1285, 632, 556, 57], [221, 610, 492, 50]], [[297, 262, 344, 126], [916, 146, 399, 56]], [[65, 108, 361, 33], [0, 0, 0, 0]], [[94, 66, 451, 672], [499, 730, 22, 7]], [[335, 159, 531, 63], [572, 128, 186, 8]], [[584, 417, 538, 102], [1248, 344, 11, 11]], [[401, 1281, 566, 61], [0, 0, 0, 0]], [[677, 220, 566, 62], [0, 0, 0, 0]], [[120, 184, 248, 26], [0, 0, 0, 0]], [[156, 564, 491, 53], [494, 562, 90, 18]], [[634, 76, 662, 73], [1152, 64, 69, 17]], [[496, 1243, 561, 55], [0, 0, 0, 0]], [[195, 767, 349, 47], [0, 0, 0, 0]], [[188, 90, 278, 50], [775, 89, 277, 39]], [[208, 156, 295, 22], [0, 0, 0, 0]], [[585, 346, 168, 25], [0, 0, 0, 0]], [[193, 454, 192, 14], [0, 0, 0, 0]], [[686, 233, 566, 61], [569, 214, 5, 6]], [[872, 343, 106, 61], [713, 311, 207, 17]], [[154, 686, 276, 17], [0, 0, 0, 0]], [[213, 173, 290, 39], [0, 0, 0, 0]], [[

In [14]:

with open('./data/text_boxes.pkl', 'wb') as handle:
    pickle.dump(contours_2, handle)

with open('./data/text_boxes.pkl', 'rb') as handle:
    b = pickle.load(handle)
print(b)

[[[549, 1653, 1101, 1702], [1568, 1647, 1589, 1658]], [[199, 548, 629, 593], [0, 0, 0, 0]], [[2770, 2022, 3370, 2108], [0, 0, 0, 0]], [[68, 464, 562, 533], [0, 0, 0, 0]], [[1285, 632, 1841, 689], [221, 610, 713, 660]], [[297, 262, 641, 388], [916, 146, 1315, 202]], [[65, 108, 426, 141], [0, 0, 0, 0]], [[94, 66, 545, 738], [499, 730, 521, 737]], [[335, 159, 866, 222], [572, 128, 758, 136]], [[584, 417, 1122, 519], [1248, 344, 1259, 355]], [[401, 1281, 967, 1342], [0, 0, 0, 0]], [[677, 220, 1243, 282], [0, 0, 0, 0]], [[120, 184, 368, 210], [0, 0, 0, 0]], [[156, 564, 647, 617], [494, 562, 584, 580]], [[634, 76, 1296, 149], [1152, 64, 1221, 81]], [[496, 1243, 1057, 1298], [0, 0, 0, 0]], [[195, 767, 544, 814], [0, 0, 0, 0]], [[188, 90, 466, 140], [775, 89, 1052, 128]], [[208, 156, 503, 178], [0, 0, 0, 0]], [[585, 346, 753, 371], [0, 0, 0, 0]], [[193, 454, 385, 468], [0, 0, 0, 0]], [[686, 233, 1252, 294], [569, 214, 574, 220]], [[872, 343, 978, 404], [713, 311, 920, 328]], [[154, 686, 430, 7

In [10]:
sim = Similarity()

qsd1_w2_similarities = sim.compute_string_similarities(detected_titles,BBDD_titles,desc='Computing string similarity', similarity='jaccard')
qsd1_w3_similarities = sim.compute_string_similarities(detected_authors,BBDD_authors,desc='Computing string similarity', similarity='jaccard')

Computing string similarity: 100%|█████████████████████████████████████████████████████| 30/30 [00:00<00:00, 79.37it/s]


In [11]:

top_k_qsd1_w2 = sim.get_top_k(qsd1_w2_similarities, db_files, 10, desc='Extracting top 10')
top_k_qsd1_w3 = sim.get_top_k(qsd1_w3_similarities, db_files, 10, desc='Extracting top 10')

Extracting top 10: 100%|████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 15035.14it/s]


In [15]:
SAVE_RESULTS =False
K=10
predicted_results_qsd1_w2 = data_manager.save_results_1(results = top_k_qsd1_w2, path = "./week3/QSD1W2/method1", save = SAVE_RESULTS) # FOR QS WITH 1 PAINTING
expected_results_qsd1_w2 = pickle.load(file = open('./data/qsd1_w2/gt_corresps.pkl', "rb"))
predicted_results_qsd1_w3 = data_manager.save_results_1(results = top_k_qsd1_w3, path = "./week3/QSD1W3/method1", save = SAVE_RESULTS) # FOR QS WITH 1 PAINTING
expected_results_qsd1_w3 = pickle.load(file = open('./data/qsd1_w3/gt_corresps.pkl', "rb"))

metric = metrics.mapk(actual = expected_results_qsd1_w2, predicted = predicted_results_qsd1_w2, k=K)
print("MAP@{} Score: {:.4f}% ({}/{})".format(K, metric*100,int(len(predicted_results_qsd1_w2)*metric),len(predicted_results_qsd1_w2)))
metric = metrics.mapk(actual = expected_results_qsd1_w3, predicted = predicted_results_qsd1_w3, k=K)
print("MAP@{} Score: {:.4f}% ({}/{})".format(K, metric*100,int(len(predicted_results_qsd1_w3)*metric),len(predicted_results_qsd1_w3)))

MAP@10 Score: 69.4444% (20/30)
MAP@10 Score: 37.0370% (11/30)


In [13]:
K=5
predicted_results_qsd1_w2 = data_manager.save_results_1(results = top_k_qsd1_w2, path = "./week3/QSD1W2/method1", save = SAVE_RESULTS) # FOR QS WITH 1 PAINTING
expected_results_qsd1_w2 = pickle.load(file = open('./data/qsd1_w2/gt_corresps.pkl', "rb"))
predicted_results_qsd1_w3 = data_manager.save_results_1(results = top_k_qsd1_w3, path = "./week3/QSD1W3/method1", save = SAVE_RESULTS) # FOR QS WITH 1 PAINTING
expected_results_qsd1_w3 = pickle.load(file = open('./data/qsd1_w3/gt_corresps.pkl', "rb"))

metric = metrics.mapk(actual = expected_results_qsd1_w2, predicted = predicted_results_qsd1_w2, k=K)
print("MAP@{} Score: {:.4f}% ({}/{})".format(K, metric*100,int(len(predicted_results_qsd1_w2)*metric),len(predicted_results_qsd1_w2)))
metric = metrics.mapk(actual = expected_results_qsd1_w3, predicted = predicted_results_qsd1_w3, k=K)
print("MAP@{} Score: {:.4f}% ({}/{})".format(K, metric*100,int(len(predicted_results_qsd1_w3)*metric),len(predicted_results_qsd1_w3)))

MAP@5 Score: 69.4444% (20/30)
MAP@5 Score: 36.1111% (10/30)


In [14]:
K=1
predicted_results_qsd1_w2 = data_manager.save_results_1(results = top_k_qsd1_w2, path = "./week3/QSD1W2/method1", save = SAVE_RESULTS) # FOR QS WITH 1 PAINTING
expected_results_qsd1_w2 = pickle.load(file = open('./data/qsd1_w2/gt_corresps.pkl', "rb"))
predicted_results_qsd1_w3 = data_manager.save_results_1(results = top_k_qsd1_w3, path = "./week3/QSD1W3/method1", save = SAVE_RESULTS) # FOR QS WITH 1 PAINTING
expected_results_qsd1_w3 = pickle.load(file = open('./data/qsd1_w3/gt_corresps.pkl', "rb"))

metric = metrics.mapk(actual = expected_results_qsd1_w2, predicted = predicted_results_qsd1_w2, k=K)
print("MAP@{} Score: {:.4f}% ({}/{})".format(K, metric*100,int(len(predicted_results_qsd1_w2)*metric),len(predicted_results_qsd1_w2)))
metric = metrics.mapk(actual = expected_results_qsd1_w3, predicted = predicted_results_qsd1_w3, k=K)
print("MAP@{} Score: {:.4f}% ({}/{})".format(K, metric*100,int(len(predicted_results_qsd1_w3)*metric),len(predicted_results_qsd1_w3)))

MAP@1 Score: 66.6667% (20/30)
MAP@1 Score: 20.0000% (6/30)
